# Dissatisfaction Levels of Former School Employees - Part 1

In this exercise, I will be cleaning two exit surveys from employees of the Department of Education, Training, and Employment (DETE) and the Technical and Further Education (TAFE) in Queensland, Australia and aftwards be combining them into a single dataset. You can download the [DETE](http://opendata.dete.qld.gov.au/human-resources/dete-exit-survey-january-2014.csv) exit survey and the [TAFE](http://opendata.dete.qld.gov.au/human-resources/tafe-employee-exit-survey-access-database-december-2013.csv) survey by clicking the links.

## 1. Reading the Dataset and Removing Unnecessary Columns

First I will import the appropriate modules, read in the dataset, and take a look at the arrangement of data as well as determine which columns are likley to be pertinent to my investigation. Afterwards, I will be better suited to cleaning the dataset to best serve my goals.

In [1]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dete_survey = pd.read_csv("dete_survey.csv", encoding="cp1252", na_values="Not Stated")
tafe_survey = pd.read_csv("tafe_survey.csv", encoding="cp1252")

### a) First Look at the Dataset

Now that I have imported the data and read in the DETE and TAFE surveys, I will take a look at the columns in each of the datsets to determine which ones are likley to be useful for my investigation.

In [2]:
dete_survey.columns

Index(['ID', 'SeparationType', 'Cease Date', 'DETE Start Date',
       'Role Start Date', 'Position', 'Classification', 'Region',
       'Business Unit', 'Employment Status', 'Career move to public sector',
       'Career move to private sector', 'Interpersonal conflicts',
       'Job dissatisfaction', 'Dissatisfaction with the department',
       'Physical work environment', 'Lack of recognition',
       'Lack of job security', 'Work location', 'Employment conditions',
       'Maternity/family', 'Relocation', 'Study/Travel', 'Ill Health',
       'Traumatic incident', 'Work life balance', 'Workload',
       'None of the above', 'Professional Development',
       'Opportunities for promotion', 'Staff morale', 'Workplace issue',
       'Physical environment', 'Worklife balance',
       'Stress and pressure support', 'Performance of supervisor',
       'Peer support', 'Initiative', 'Skills', 'Coach', 'Career Aspirations',
       'Feedback', 'Further PD', 'Communication', 'My say', 'Inform

The most important columns in the DETE dataset are:
- SeparationType = manner in which the employee left the school district
- DETE Start Date, Role Start Date = could be used to calculate tenure, i.e. the length of time in which the employee worked for the school district
- Position = what role the employee had for the school district
- Employment Status = if the employee was full time, part time, or other
- Career move to public sector to Worklife balance = if the employee left the school district voluntarily, these series of columns that indicate the reason as to why
- Age = age of the employee at the time of leaving

Next, I will take a look at the columns in the TAFE dataset.

In [3]:
tafe_survey.columns

Index(['Record ID', 'Institute', 'WorkArea', 'CESSATION YEAR',
       'Reason for ceasing employment',
       'Contributing Factors. Career Move - Public Sector ',
       'Contributing Factors. Career Move - Private Sector ',
       'Contributing Factors. Career Move - Self-employment',
       'Contributing Factors. Ill Health',
       'Contributing Factors. Maternity/Family',
       'Contributing Factors. Dissatisfaction',
       'Contributing Factors. Job Dissatisfaction',
       'Contributing Factors. Interpersonal Conflict',
       'Contributing Factors. Study', 'Contributing Factors. Travel',
       'Contributing Factors. Other', 'Contributing Factors. NONE',
       'Main Factor.     Which of these was the main factor for leaving?',
       'InstituteViews. Topic:1. I feel the senior leadership had a clear vision and direction',
       'InstituteViews. Topic:2. I was given access to skills training to help me do my job better',
       'InstituteViews. Topic:3. I was given adequate 

The columns that look the most important in the TAFE dataset are:
- Reason for ceasing employment = manner in which the employee left the company
- Contributing Factors = if the employee left voluntarily, series of columns that indicate the reason as to why
- CurrentAge.     Current Age = age of the employee at time of leaving
- Employment Type.     Employment Type = full time, part time, or other
- Classification.     Classification = what role the employee had in the school district
- LengthofServiceOverall. Overall Length of Service at Institute (in years) = would indicate the tenure of the employee

In both the DETE and TAFE dataset, there are a lot of columns that will not be useful to analyze, so I will remove them from the datasets.

### b) Removing Unecessary Columns

Columns 28-48 from the DETE dataset and 17-65 from the TAFE dataset are very unlikely to be useful to my analysis. Removing these columns will make it much easier to read and analyze the data. I will create two data subsets, dete_survey_updated and tafe_survey_updated, that do not contain these undesireable columns. I will use these subsets for the remainder of my analysis.

In [4]:
dete_survey_updated = dete_survey.drop(dete_survey.columns[28:49], axis=1)

In [5]:
tafe_survey_updated = tafe_survey.drop(tafe_survey.columns[17:66], axis=1)

In [6]:
print(dete_survey_updated.columns)
print("")
print(tafe_survey_updated.columns)

Index(['ID', 'SeparationType', 'Cease Date', 'DETE Start Date',
       'Role Start Date', 'Position', 'Classification', 'Region',
       'Business Unit', 'Employment Status', 'Career move to public sector',
       'Career move to private sector', 'Interpersonal conflicts',
       'Job dissatisfaction', 'Dissatisfaction with the department',
       'Physical work environment', 'Lack of recognition',
       'Lack of job security', 'Work location', 'Employment conditions',
       'Maternity/family', 'Relocation', 'Study/Travel', 'Ill Health',
       'Traumatic incident', 'Work life balance', 'Workload',
       'None of the above', 'Gender', 'Age', 'Aboriginal', 'Torres Strait',
       'South Sea', 'Disability', 'NESB'],
      dtype='object')

Index(['Record ID', 'Institute', 'WorkArea', 'CESSATION YEAR',
       'Reason for ceasing employment',
       'Contributing Factors. Career Move - Public Sector ',
       'Contributing Factors. Career Move - Private Sector ',
       'Contributing Fac

## 2. Editing the Column Names

Now that the dataset is much more readable, I can rename the columns so that both datasets contain the same column names. This will make it much easier to merge the two datasets later on.

### a) DETE Survey Dataset

In order to make the column names in the DETE database more readable, we will complete the following:
1. Change the name of SeparationType to Separation Type so it follows the naming convention of the other columns
2. Merge all the columns relating to why the employee left the company into a single column
3. Remove any other unnecessary columns
\end{itemize}

First, I will rename the SeparationType column.

In [7]:
dete_survey_updated = dete_survey_updated.rename(columns={"SeparationType": "Separation Type"})

Now I will merge all columns that contain reasons for why an employee voluntarily left the school district, which range from "Career move to public sector" to "None of the above". The data entries in these columns contain the boolean value True if the reason in the column title influenced the employee's decicion to leave and contain the boolean value False if it did not. Thus, I will combine all of the columns into a single column containing all of the reasons as to why the employee left.

In [8]:
# Contains all of the values in the dataset that indicate why the employee left the company
reasons_for_leaving = ['Career move to public sector',
                       'Career move to private sector',
                       'Interpersonal conflicts',
                       'Job dissatisfaction',
                       'Dissatisfaction with the department',
                       'Physical work environment',
                       'Lack of recognition',
                       'Lack of job security',
                       'Work location',
                       'Employment conditions',
                       'Maternity/family',
                       'Relocation',
                       'Study/Travel',
                       'Ill Health',
                       'Traumatic incident',
                       'Work life balance',
                       'Workload',
                       'None of the above']

# DETE dataset containing just the columns in reasons_for_leaving
dete_survey_updated[reasons_for_leaving].head(3)

,Career move to public sector,Career move to private sector,Interpersonal conflicts,Job dissatisfaction,Dissatisfaction with the department,Physical work environment,Lack of recognition,Lack of job security,Work location,Employment conditions,Maternity/family,Relocation,Study/Travel,Ill Health,Traumatic incident,Work life balance,Workload,None of the above
0,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


Next I will create a function creates a string that lists all of the reasons as to why the employee left the district. If the value within a given column is True, I will add the name of the column (and thus the factor influencing the employee to leave) to the list of new values. I will place this list in a new column, which I will name Reasons for Leaving.

In [9]:
def merge_reasons(row, columns_to_merge):
    column_names = dete_survey_updated[columns_to_merge].columns # names of all columns for reasons why the employee left
    reasons = row[columns_to_merge] # all values for reasons_for_leaving in the given row
    merged = [] # will contain all reasons the employee left the company
    merged_string = "" # will contain a string of all reasons the employee left the company
    
    column_name_index = 0 # will be used to determine index of column name
    for reason in reasons:
        if reason == True:
            merged.append(column_names[column_name_index]) # adds column name to the merged list if column is True
        column_name_index += 1

    merged = sorted(merged) # sorts reasons alphabetically
    
    if len(merged) > 0:
        for reason in merged:
            merged_string += reason # adds all values in merged to the new string
            if merged[-1] != reason:
                merged_string += ", " # creates a comma in between values
                
    else:
        merged_string += "Not Specified" # if every column is True, will set the string to "Not Specified"
        
    merged_string = merged_string.replace("None of the above", "Not Specified") # ensures all "None of the above" values become "Not Specified" values
        
    return merged_string

In [10]:
dete_survey_updated["Reasons for Leaving"] = dete_survey_updated.apply(merge_reasons, axis=1, args=([reasons_for_leaving]))
dete_survey_updated["Reasons for Leaving"]


0      Career move to public sector, Job dissatisfact...
1                                          Not Specified
2                                          Not Specified
3                          Career move to private sector
4                                      Work life balance
                             ...                        
817               Ill Health, Maternity/family, Workload
818                                        Not Specified
819      Maternity/family, Relocation, Work life balance
820                                           Ill Health
821                                        Not Specified
Name: Reasons for Leaving, Length: 822, dtype: object

Now that I have merged all of the reasons for leaving into a single column, I can remove all of the columns in the dataset that have now been merged into the Reasons for Leaving column.

In [11]:
dete_survey_updated = dete_survey_updated.drop(reasons_for_leaving, axis=1)

I'm also going to remove the columns in the range "Aboriginal" to "NESB" as they don't seem to be pertinent to my analysis. The code below will remove those columns from our dataset.

In [12]:
dete_survey_updated = dete_survey_updated.drop(dete_survey_updated.columns[12:17], axis=1)
dete_survey_updated.columns

Index(['ID', 'Separation Type', 'Cease Date', 'DETE Start Date',
       'Role Start Date', 'Position', 'Classification', 'Region',
       'Business Unit', 'Employment Status', 'Gender', 'Age',
       'Reasons for Leaving'],
      dtype='object')

Now that the columns in DETE dataset have been cleaned, I can begin working on renaming the columns in the TAFE dataset.

### b) TAFE Survey Dataset

In order to rename the column in the TAFE dataset so it's easier to analyze, I will complete the following steps:
1. Remove all unnecessay whitespace from the column names
2. Merge the Contributing Factors columns into a single column
3. Rename the other columns so that they all adhere to the same naming convention

First, I will strip away all of the whitespace in all of the column names.

In [13]:
tafe_survey_updated.columns = tafe_survey_updated.columns.str.strip()
tafe_survey_updated.columns

Index(['Record ID', 'Institute', 'WorkArea', 'CESSATION YEAR',
       'Reason for ceasing employment',
       'Contributing Factors. Career Move - Public Sector',
       'Contributing Factors. Career Move - Private Sector',
       'Contributing Factors. Career Move - Self-employment',
       'Contributing Factors. Ill Health',
       'Contributing Factors. Maternity/Family',
       'Contributing Factors. Dissatisfaction',
       'Contributing Factors. Job Dissatisfaction',
       'Contributing Factors. Interpersonal Conflict',
       'Contributing Factors. Study', 'Contributing Factors. Travel',
       'Contributing Factors. Other', 'Contributing Factors. NONE',
       'Gender.     What is your Gender?', 'CurrentAge.     Current Age',
       'Employment Type.     Employment Type',
       'Classification.     Classification',
       'LengthofServiceOverall. Overall Length of Service at Institute (in years)',
       'LengthofServiceCurrent. Length of Service at current workplace (in year

Using the strip method did not remove the spacing between the column names. There must be some character other than a space that is causing this spacing issue. I wuse a loop to take a look at all of the charcters in the "Gender.     What is your Gender?" column (which has this strange spacing issue) so I can determine what this strage character is and remove it.

In [14]:
test_char = []
for char in tafe_survey_updated.columns[17]:
    test_char.append(char)
print(test_char)

['G', 'e', 'n', 'd', 'e', 'r', '.', ' ', '\xa0', '\xa0', '\xa0', '\xa0', 'W', 'h', 'a', 't', ' ', 'i', 's', ' ', 'y', 'o', 'u', 'r', ' ', 'G', 'e', 'n', 'd', 'e', 'r', '?']


There is a strange char that is causing the additional spacing between the column names. I will replace this char with an empty string in order to remove this spacing issue.

In [15]:
tafe_survey_updated.columns = tafe_survey_updated.columns.str.replace('\xa0', "")
tafe_survey_updated.columns

Index(['Record ID', 'Institute', 'WorkArea', 'CESSATION YEAR',
       'Reason for ceasing employment',
       'Contributing Factors. Career Move - Public Sector',
       'Contributing Factors. Career Move - Private Sector',
       'Contributing Factors. Career Move - Self-employment',
       'Contributing Factors. Ill Health',
       'Contributing Factors. Maternity/Family',
       'Contributing Factors. Dissatisfaction',
       'Contributing Factors. Job Dissatisfaction',
       'Contributing Factors. Interpersonal Conflict',
       'Contributing Factors. Study', 'Contributing Factors. Travel',
       'Contributing Factors. Other', 'Contributing Factors. NONE',
       'Gender. What is your Gender?', 'CurrentAge. Current Age',
       'Employment Type. Employment Type', 'Classification. Classification',
       'LengthofServiceOverall. Overall Length of Service at Institute (in years)',
       'LengthofServiceCurrent. Length of Service at current workplace (in years)'],
      dtype='obje

Removing the strange char seemed to remedy the spacing issue. Now, I can merge all of the "Contributing Factors" columns into a single Reasons for Leaving column so that a single column will a list of all of the reasons as to why a given person decided to leave the school district. First, I will take a look at all of the columns that contain "Contributing Factors" within the title name.

In [16]:
contributing_factors = []

for column in tafe_survey_updated.columns:
    if "Contributing Factors" in column:
        contributing_factors.append(column)
        
tafe_survey_updated[contributing_factors].head(3)

,Contributing Factors. Career Move - Public Sector,Contributing Factors. Career Move - Private Sector,Contributing Factors. Career Move - Self-employment,Contributing Factors. Ill Health,Contributing Factors. Maternity/Family,Contributing Factors. Dissatisfaction,Contributing Factors. Job Dissatisfaction,Contributing Factors. Interpersonal Conflict,Contributing Factors. Study,Contributing Factors. Travel,Contributing Factors. Other,Contributing Factors. NONE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-,-,-,-,-,-,-,-,-,Travel,-,-
2,-,-,-,-,-,-,-,-,-,-,-,NONE


The Contribution Factors columns have two possible values: 
- Dash (-) = indicates the factor does not apply
- Name of Title = indicates the factor does apply

I will merge all of these Contributing Factors columns into a single column that contains all of the reasons the employee left the school district. First, I will create a Reasons for Leaving column that will eventually contain a list of all reasons the employee left. Then, I will create a function that adds all of the contributing factors to the list, which will be in a string format, in a similar manner as I did for the DETE dataset.

In [17]:
def merge_factors(row):
    reasons = row[contributing_factors] # all values for reasons_for_leaving in the given row
    merged = [] # will contain all reasons the employee left the company
    merged_string = "" # will contain a string of all reasons the employee left the company
    
    for reason in reasons:
        if reason != "-":
            merged.append(reason) # adds column name to the merged list if column is True

    merged = sorted(merged) # sorts reasons alphabetically
    
    if len(merged) > 0:
        for reason in merged:
            try:
                merged_string += reason # adds all values in merged to the new string
                if merged[-1] != reason:
                    merged_string += ", " # creates a comma in between values
            except TypeError:
                merged += ""
    
    merged_string = merged_string.replace(", NONE", "")
    merged_string = merged_string.replace(", Other", "")
    merged_string = merged_string.replace("NONE", "")
    merged_string = merged_string.replace("Other", "")
                
    if merged_string == "":
        merged_string = "Not Specified" # if every column is True, will set the string to "Not Specified"
    
    return merged_string

In [18]:
tafe_survey_updated["Reasons for Leaving"] = tafe_survey_updated.apply(merge_factors, axis=1)
tafe_survey_updated["Reasons for Leaving"]

0                              Not Specified
1                                     Travel
2                              Not Specified
3                                     Travel
4               Career Move - Private Sector
                       ...                  
697              Career Move - Public Sector
698              Career Move - Public Sector
699                            Not Specified
700                            Not Specified
701    Career Move - Self-employment, Travel
Name: Reasons for Leaving, Length: 702, dtype: object

And now that I have merged all of the contributing factors into a single column, I will remove all of the Contributing Factor columns from the dataset.

In [19]:
tafe_survey_updated = tafe_survey_updated.drop(contributing_factors, axis=1)

And finally, now that all of the contributing factors have been merged into a single column, I will rename some of the columns in the dataset so they fit a better naming convention and are more readable.

In [20]:
# Update column names to match the names in dete_survey_updated
mapping = {'Record ID': 'ID', 
           'WorkArea': "Work Area",
           'CESSATION YEAR': 'Cease Date', 
           'Reason for ceasing employment': 'Separation Type', 
           'Gender. What is your Gender?': 'Gender', 
           'CurrentAge. Current Age': 'Age',
           'Employment Type. Employment Type': 'Employment Status',
           'Classification. Classification': 'Position',
           'LengthofServiceOverall. Overall Length of Service at Institute (in years)': 'Institute Service',
           'LengthofServiceCurrent. Length of Service at current workplace (in years)': 'Role Service'
          }
tafe_survey_updated = tafe_survey_updated.rename(mapping, axis = 1)

# Check that the specified column names were updated correctly
tafe_survey_updated.columns

Index(['ID', 'Institute', 'Work Area', 'Cease Date', 'Separation Type',
       'Gender', 'Age', 'Employment Status', 'Position', 'Institute Service',
       'Role Service', 'Reasons for Leaving'],
      dtype='object')

In [21]:
dete_survey_updated.columns

Index(['ID', 'Separation Type', 'Cease Date', 'DETE Start Date',
       'Role Start Date', 'Position', 'Classification', 'Region',
       'Business Unit', 'Employment Status', 'Gender', 'Age',
       'Reasons for Leaving'],
      dtype='object')

Eventually, I will ensure that both the TAFE and DETE datasets have the same column names (both in quantity and actual name themselves), but both datasets are clean enough for now.

## 3. Resigning and Dissatisfied Employees

Before I begin adjusting the possible values within each dataset so they are the same in both, I will first have to do the following:
1. Limit the dataset to only data entries of employees who resigned
2. Create an "Institute Service" column in the DETE dataset that indicates tenure each employee served
3. Find out which employees resigned due to dissatisfaction

### a) Employees Who Resigned

Note that since I'm interested in dissatisfaction levels amongst employees who left the company, I am only interested in analyzing the data from employees who voluntarily resigned. While employees who were fired or retired may have been dissatisfied , they did not leave the district for those reason. In order to find out which employees resigned, I will analyze the Separation Type column in each of the datasets.

In [22]:
dete_survey_updated["Separation Type"].value_counts()

Age Retirement                          285
Resignation-Other reasons               150
Resignation-Other employer               91
Resignation-Move overseas/interstate     70
Voluntary Early Retirement (VER)         67
Ill Health Retirement                    61
Other                                    49
Contract Expired                         34
Termination                              15
Name: Separation Type, dtype: int64

In [23]:
tafe_survey_updated["Separation Type"].value_counts()

Resignation                 340
Contract Expired            127
Retrenchment/ Redundancy    104
Retirement                   82
Transfer                     25
Termination                  23
Name: Separation Type, dtype: int64

In the DETE dataset, there are three different types of resignations while in the TAFE dataset, there is only one type of resignation. I will create a subset for each dataset that contains only data entries of employees that resigned.

In [24]:
resignation_bool = (dete_survey_updated["Separation Type"] == "Resignation-Other reasons") | (dete_survey_updated["Separation Type"] == "Resignation-Other employer") | (dete_survey_updated["Separation Type"] == "Resignation-Move overseas/interstate")
dete_resignations = dete_survey_updated[resignation_bool]
tafe_resignations = tafe_survey_updated[tafe_survey_updated["Separation Type"] == "Resignation"]

In [25]:
print(len(dete_resignations) / len(dete_survey_updated))

0.37834549878345497


In [26]:
print(len(tafe_resignations) / len(tafe_survey_updated))

0.4843304843304843


It looks as though employees are much more likely to resign from TAFE (over 48%) than they are from DETE (less than 38%). Now that all the data entries contain employees who resigned, I will drop the Separation Type column from each of the data subsets.

In [27]:
dete_resignations = dete_resignations.drop("Separation Type", axis=1)
tafe_resignations = tafe_resignations.drop("Separation Type", axis=1)

I will use dete_resignations and tafe_resignations for the remainer of my analysis.

### b) Cease Dates

Although the TAFE dataset has a value that contains information on length of tenure for employees, the DETE dataset does not. However, it does have two columns (DETE Start Date and Cease Date) that could be used to calculate this information. Let's take a look at both of those columns.

In [28]:
dete_resignations["DETE Start Date"].value_counts().head()

2011.0    24
2008.0    22
2007.0    21
2012.0    21
2010.0    17
Name: DETE Start Date, dtype: int64

In [29]:
dete_resignations["Cease Date"].value_counts().head()

2012       126
2013        74
01/2014     22
12/2013     17
06/2013     14
Name: Cease Date, dtype: int64

While  the DETE Start Date column only contains only years, the "Cease Date" column contains both years and months. Since I'm only interested in the year (I just need the number of years the employee has worked, anything more specific than that would be unecessary), I will extract the year from the Cease Date column and replace the value with the year.

In [30]:
dete_resignations['Cease Date'] = dete_resignations['Cease Date'].str.split("/").str[-1]

In [31]:
dete_resignations["Cease Date"].value_counts()

2013    146
2012    129
2014     22
2010      2
2006      1
Name: Cease Date, dtype: int64

Now that I have extracted the years from the Cease Date column, I will be able to subtract the DETE Start Date column from the Cease Date column to calculate the employees' length of tenure in years.

### c) Tenure Worked

While the TAFE dataset has a column called Institute Service that indicates tenure, the DETE dataset does not have a similar column. However, subtracting the DETE Start Date from the Cease Date columns I will be able to create a similar column in the DETE dataset.

In [32]:
tafe_resignations["Institute Service"].value_counts()

Less than 1 year      73
1-2                   64
3-4                   63
5-6                   33
11-20                 26
7-10                  21
More than 20 years    10
Name: Institute Service, dtype: int64

The Institute Service column contains a series of bins with a range of values that indicate length of tenure. For example, the "1-2" bin would contain all employees who worked for 1-2 years before resigning. I will create a function that calculates the tenure for each of the employees and then place each employee in one of seven bins. I will use this formula to create an Institute Service column in the DETE resignations dataset.

In [33]:
def tenure_finder(row):
    tenure = ""
    try:
        tenure = int(row["Cease Date"]) - int(row["DETE Start Date"])

        if tenure < 1:
            return "Less than 1 year"
        elif tenure <= 2:
            return "1-2"
        elif tenure <= 4:
            return "3-4"         
        elif tenure <= 6:
            return "5-6" 
        elif tenure <= 10:
            return "7-10" 
        elif tenure <= 20:
            return "11-20" 
        else:
            return "More than 20 years"
    
    except ValueError:
        next
                 
dete_resignations["Institute Service"] = dete_resignations.apply(tenure_finder, axis=1)
dete_resignations["Institute Service"].value_counts()

11-20                 57
More than 20 years    43
7-10                  41
5-6                   40
1-2                   36
3-4                   36
Less than 1 year      20
Name: Institute Service, dtype: int64

Now that the DETE "Institute Service" column has been created, note the average tenure for the employees in the DETE resignations dataset is much higher than that of the employees in the TAFE resignations dataset.

### d) Dissatisfaction Column

There are several values in the "Reasons for Leaving" columns of the DETE and TAFE surveys that indicate dissatisfaction:
- DETE Survey:
    - Job dissatisfaction
    - Dissatisfaction with the department
    - Physical work environment
    - Lack of Recognition
    - Lack of Job Security
    - Work location
    - Employment conditions
    - Work life balance
    - Workload
- TAFE Survey:
    - Contributing Factors. Dissatisfaction
    - Job Dissatisfaction

I will use this data to create a column that indicates whether the person resigned due to dissatisfaction (via the categories above) with a Boolean True or False value.

In [34]:
dete_dissatisfaction = ["Job dissatisfaction",
                        "Dissatisfaction with the department",
                        "Physical work environment",
                        "Lack of recognition",
                        "Lack of job security",
                        "Work location",
                        "Employment conditions", 
                        "Work life balance",
                        "Workload"]

tafe_dissatisfaction = ["Contributing Factors. Dissatisfaction ", "Job Dissatisfaction"]

In [35]:
def dissatisfaction(row, diss_values):
    contributing_factors = row["Reasons for Leaving"].split(", ")
    
    for value in contributing_factors:
        if value in diss_values:
            return True
        
    return False

In [36]:
dete_resignations["Dissatisfaction"] = dete_resignations.apply(dissatisfaction, axis=1, args=([dete_dissatisfaction]))
dete_resignations[["Reasons for Leaving", "Dissatisfaction"]].head()

,Reasons for Leaving,Dissatisfaction
3,Career move to private sector,False
5,"Career move to private sector, Employment cond...",True
8,Career move to private sector,False
9,"Dissatisfaction with the department, Interpers...",True
11,"Maternity/family, Relocation",False


In [37]:
tafe_resignations["Dissatisfaction"] = tafe_resignations.apply(dissatisfaction, axis=1, args=([tafe_dissatisfaction]))
tafe_resignations[["Reasons for Leaving", "Dissatisfaction"]].head(5)

,Reasons for Leaving,Dissatisfaction
3,Travel,False
4,Career Move - Private Sector,False
5,Not Specified,False
6,"Career Move - Private Sector, Maternity/Family",False
7,Not Specified,False


Next I will create a function that replaces all of the values that indicate dissatisfaction in the Reasons for Leaving columns of both datasets with the word Dissatisfaction. If a reason indicating dissatisfaction appears more than once in the data entry, the function will eliminate the duplicates.

In [38]:
def dissatisfaction_organizer(row, diss_values):
    reasons_for_leaving = row["Reasons for Leaving"].split(", ")
    
    reasons = []
    reasons_string = ""
    
    for value in reasons_for_leaving:
        if value in diss_values:
            reasons_for_leaving.append("Dissatisfaction")
            
    for reason in reasons_for_leaving:    
        if reason not in diss_values and (reason != "Dissatisfaction" or "Dissatisfaction" not in reasons):
            reasons.append(reason)
            
    reasons = sorted(reasons)
        
    for reason in reasons:
        reasons_string += reason
        if reasons[-1] != reason:
            reasons_string += ", "
    
               
    return reasons_string

In [39]:
dete_resignations["Reasons for Leaving"] = dete_resignations.apply(dissatisfaction_organizer, axis=1, args=([dete_dissatisfaction]))
dete_resignations[["Reasons for Leaving", "Dissatisfaction"]].head()


,Reasons for Leaving,Dissatisfaction
3,Career move to private sector,False
5,"Career move to private sector, Dissatisfaction...",True
8,Career move to private sector,False
9,"Dissatisfaction, Interpersonal conflicts",True
11,"Maternity/family, Relocation",False


In [40]:
tafe_resignations["Reasons for Leaving"] = tafe_resignations.apply(dissatisfaction_organizer, axis=1, args=([tafe_dissatisfaction]))
tafe_resignations[tafe_resignations["Dissatisfaction"] == True][["Reasons for Leaving", "Dissatisfaction"]].head()


,Reasons for Leaving,Dissatisfaction
14,"Career Move - Private Sector, Dissatisfaction",True
17,"Career Move - Public Sector, Dissatisfaction",True
20,Dissatisfaction,True
26,"Career Move - Private Sector, Dissatisfaction",True
40,Dissatisfaction,True


Now that I can easily tell which employees resigned due to dissatisfaction, I will work on making sure both datasets have the exact same columns and that the possible values in each of those columns are the same. This will allow me to merge the datasets together and to make comparisons between the two datasets much easier.

## 4. Preparing the Datasets for a Merge

In order to concatinate the datasets, I first have to ensure  both datasets have the same columns and that all possible values within each column are the same.

### a) Editing the Column Names

First, I will take a look at the names of the columns in each of the datasets to make sure they are the same.

In [41]:
dete_resignations.columns

Index(['ID', 'Cease Date', 'DETE Start Date', 'Role Start Date', 'Position',
       'Classification', 'Region', 'Business Unit', 'Employment Status',
       'Gender', 'Age', 'Reasons for Leaving', 'Institute Service',
       'Dissatisfaction'],
      dtype='object')

In [42]:
tafe_resignations.columns

Index(['ID', 'Institute', 'Work Area', 'Cease Date', 'Gender', 'Age',
       'Employment Status', 'Position', 'Institute Service', 'Role Service',
       'Reasons for Leaving', 'Dissatisfaction'],
      dtype='object')

It appears as though both datasets have column names that the other dataset does not have. I will get rid of all of these columns as they don't seem pertinent to my investigation. Additionally, I will drop the ID and Cease Date from both datasets as they are also not very useful.

In [43]:
dete_resignations = dete_resignations.drop(["ID", "DETE Start Date", "Role Start Date", "Region", "Business Unit", "Classification", "Cease Date"], axis=1)
tafe_resignations = tafe_resignations.drop(["ID", "Institute", "Work Area", "Role Service", "Cease Date"], axis=1)


In [44]:
dete_resignations.columns

Index(['Position', 'Employment Status', 'Gender', 'Age', 'Reasons for Leaving',
       'Institute Service', 'Dissatisfaction'],
      dtype='object')

In [45]:
tafe_resignations.columns

Index(['Gender', 'Age', 'Employment Status', 'Position', 'Institute Service',
       'Reasons for Leaving', 'Dissatisfaction'],
      dtype='object')

Now both of the datasets only have seven columns each and they are the same in both datasets. Now I will make sure that all possible values within each column name are the same as well.

### b) Editing Column Values

Before I begin looking at the values within each column, I will create a simple function that uses a map in order to change values within a given column into new values.

In [46]:
def value_changer(col_map, col):
    for key, value in col_map.items():
        tafe_resignations.loc[tafe_resignations[col] == key, col] = value
        dete_resignations.loc[dete_resignations[col] == key, col] = value

Next, I will take a look at the possible values within the Gender column. I suspect they will be the same in both datasets, but I will check to make sure.

In [47]:
dete_resignations["Gender"].value_counts()

Female    233
Male       69
Name: Gender, dtype: int64

In [48]:
tafe_resignations["Gender"].value_counts()

Female    191
Male       99
Name: Gender, dtype: int64

The Gender columns doesn't need adjusting, so I will move on to the Age column.

In [49]:
dete_resignations["Age"].value_counts()

41-45            48
46-50            42
36-40            41
26-30            35
51-55            32
31-35            29
21-25            29
56-60            26
61 or older      23
20 or younger     1
Name: Age, dtype: int64

In [50]:
tafe_resignations["Age"].value_counts()

41 – 45          45
51-55            39
46 – 50          39
21 – 25          33
36 – 40          32
31 – 35          32
26 – 30          32
56 or older      29
20 or younger     9
Name: Age, dtype: int64

While the possible values in the Age column are very similar in both datasets, there are a few notable differences:
- DETE dataset has a "56-60" category and a "61 or older" category, whereas the TAFE dataset has a "56 or older" category that contains both groups
- The TAFE dataset has additional spacings between the hyphens for most of the values, for example "41 – 45" should be "41-45"

I will use the value_changer function in order to remedy these values. Additionally, I will use the function to change "56 or older" to "Over 55" and "20 or younger" to "Under 21".

In [51]:
age_map = {"20 or younger": "18-20",
           "21 – 25": "21-25",
           "26 – 30": "26-30",
           "31 – 35": "31-35",
           "36 – 40": "36-40",
           "41 – 45": "41-45",
           "46 – 50": "46-50",
           "56-60": "56+",
           "56 or older": "56+",
           "61 or older": "56+"
}

value_changer(age_map, "Age")

In [52]:
dete_resignations["Age"].value_counts()

56+      49
41-45    48
46-50    42
36-40    41
26-30    35
51-55    32
31-35    29
21-25    29
18-20     1
Name: Age, dtype: int64

In [53]:
tafe_resignations["Age"].value_counts()

41-45    45
51-55    39
46-50    39
21-25    33
31-35    32
36-40    32
26-30    32
56+      29
18-20     9
Name: Age, dtype: int64

Now that both datasets have the same nine possible values in the Age column, I will take a look at the possible values for the Employment Status column.

In [54]:
dete_resignations["Employment Status"].value_counts()

Permanent Full-time    158
Permanent Part-time    125
Temporary Part-time     10
Temporary Full-time      9
Casual                   5
Name: Employment Status, dtype: int64

In [55]:
tafe_resignations["Employment Status"].value_counts()

Temporary Full-time    111
Permanent Full-time     98
Contract/casual         29
Temporary Part-time     27
Permanent Part-time     25
Name: Employment Status, dtype: int64

The DETE dataset has a "Casual" value whereas the TAFE dataset has a "Contract/casual" value.  I will rename the "Contract/casual" values in the TAFE dataset to "Casual" and additionally shorten the names of the rest of the values in the dataset using the value_changer function. 

In [56]:
def value_changer(col_map, col):
    for key, value in col_map.items():
        tafe_resignations.loc[tafe_resignations[col] == key, col] = value
        dete_resignations.loc[dete_resignations[col] == key, col] = value

In [57]:
employment_map = {"Contract/casual": "Casual",
                  "Temporary Full-time": "Temp (Part)",
                  "Permanent Full-time": "Full-time",
                  "Temporary Part-time": "Temp (Full)",
                  "Permanent Part-time": "Part-time"
}

value_changer(employment_map, "Employment Status")       
tafe_resignations["Employment Status"].value_counts()

Temp (Part)    111
Full-time       98
Casual          29
Temp (Full)     27
Part-time       25
Name: Employment Status, dtype: int64

Next, I will examine the possible values of the Position columns.

In [58]:
dete_resignations["Position"].value_counts()

Teacher                                                    129
Teacher Aide                                                63
Cleaner                                                     39
Public Servant                                              30
Head of Curriculum/Head of Special Education                10
School Administrative Staff                                  8
Technical Officer                                            8
Schools Officer                                              7
School Based Professional Staff (Therapist, nurse, etc)      5
Guidance Officer                                             3
Other                                                        3
Professional Officer                                         2
Business Service Manager                                     1
Name: Position, dtype: int64

In [59]:
tafe_resignations["Position"].value_counts()

Administration (AO)           148
Teacher (including LVT)        95
Professional Officer (PO)      16
Operational (OO)               13
Workplace Training Officer      6
Technical Officer (TO)          5
Executive (SES/SO)              4
Tutor                           3
Name: Position, dtype: int64

There are a lot of differences between the possible values for this column in each dataset, so I will need to significantly adjust the names of the possible values. First, I will create a new group of possible values that encompases all of the values within each dataset. Then, I will use these new values to replace the old values in each dataset. These new values (as well as the values they will replace) are listed on the table below.

| New Value | TAFE | DETE |
|:----|:--------|:----------|
| Administration | Administration (AO), Executive (SES/SO) | School Administrative Staff, Head of Curriculum/Head of Special Education, Business Service Manager, Public Servant |
| Teacher / Teacher's Aide | Teacher (including LVT) | Teacher, Teacher Aide |
| Officer | Professional Officer (PO), Workplace Training Officer, Technical Officer (TO) | Technical Officer, Schools Officer, Guidance Officer, Professional Officer |
| Operational | Operational (OO) | Cleaner |
| Other | Tutor | Other, School Based Professional Staff (Therapist, nurse, etc) |

Now, I will create a map and use the position_changer function to change all of the old values to these new values.

In [60]:
position_map = {"Administration (AO)": "Administration",
                "Executive (SES/SO)": "Administration",
                "School Administrative Staff": "Administration",
                "Head of Curriculum/Head of Special Education": "Administration",
                "Business Service Manager": "Administration",
                "Public Servant": "Administration",
                
                "Teacher (including LVT)": "Teacher / TA",
                "Teacher": "Teacher / TA",
                "Teacher Aide": "Teacher / TA",
                
                "Professional Officer (PO)": "Officer",
                "Workplace Training Officer": "Officer",
                "Technical Officer (TO)": "Officer",
                "Technical Officer": "Officer",
                "Schools Officer": "Officer",
                "Guidance Officer": "Officer",
                "Professional Officer": "Officer",
                
                "Operational (OO)": "Operations",
                "Cleaner": "Operations",
                
                "Tutor": "Other",
                "School Based Professional Staff (Therapist, nurse, etc)": "Other"
}

value_changer(position_map, "Position")

In [61]:
dete_resignations["Position"].value_counts()

Teacher / TA      192
Administration     49
Operations         39
Officer            20
Other               8
Name: Position, dtype: int64

In [62]:
tafe_resignations["Position"].value_counts()

Administration    152
Teacher / TA       95
Officer            27
Operations         13
Other               3
Name: Position, dtype: int64

Note that the TAFE dataset has a very large number of "Administration" workers whereas the DETE dataset has a very large number of "Teacher / Teacher's Aide" workers.

Note that the Institute Service columns should be identical in each dataset since I modeled the Institute Service column in the DETE dataset after the one in the TAFE dataset. However, I'm going to rename the Institute Service column to Institute Tenure so it's more self-explanatory as to the column's purpose. I will also use the value_changer function to change the value "Less than 1 year" to "Under 1" and the value "More than 20 years" to "Over 20".

In [63]:
dete_resignations = dete_resignations.rename(columns={"Institute Service": "Institute Tenure"})
tafe_resignations = tafe_resignations.rename(columns={"Institute Service": "Institute Tenure"})

tenure_map = {"Less than 1 year": "0-1",
              "More than 20 years": "21+"
}

value_changer(tenure_map, "Institute Tenure")

dete_resignations["Institute Tenure"].value_counts()

11-20    57
21+      43
7-10     41
5-6      40
1-2      36
3-4      36
0-1      20
Name: Institute Tenure, dtype: int64

Now I will move on to the "Reasons for Leaving" column, which should be fairly complicated since it contains a list of all possible values rather than a single value. I will first need to create a dictionary containing new values that encompass all of the values in each dataset. Then, I will need to create a function that converts the old values in both datasets to these new values. 

In [64]:
transform = {
                    "Career move to public sector": "Career Move to Public Sector",
                    "Career move to private sector": "Career Move to Private Sector",
                    "Interpersonal conflicts": "Interpersonal Conflict",
                    "Maternity/family": "Maternal/Family",
                    "Traumatic incident": "Ill Health",
                    "Career Move - Public Sector": "Career Move to Public Sector",
                    "Career Move - Private Sector": "Career Move to Private Sector",
                    "Career Move - Self-employment": "Career Move to Self-employment",
                    "Study": "Study/Travel",
                    "Travel": "Study/Travel"
                }

Next, I will create the function that converts the current values in the Reasons for Leaving column in each dataset into these newly defined values.

In [65]:
def transformer(row):
    contributing_factors = row["Reasons for Leaving"].split(", ")
    result = []
    new_result = ""
    
    # Converts the old values into the new values and adds them to the contributing_factors list
    for value in contributing_factors:
        if value in transform:
            contributing_factors.append(transform[value])
    
    # Note contributing_factors contains both new and old values
    # Only adds new values to the list, does not add old values
    if len(contributing_factors) > 0:
        for factor in contributing_factors:
            if factor not in result and factor not in transform:
                result.append(factor)
    
    # If the list is empty, makes the value "Not Specified"
    else:
        result.append("Not Specified")
    
    # Sorts the results alphabetically
    result = sorted(result)
    
    # Converts the list result to a string list
    for factor in result:
        new_result += factor
        if result[-1] != factor:
            new_result += ", "
    
    return new_result

In [66]:
dete_resignations["Reasons for Leaving"] = dete_resignations.apply(transformer, axis=1)

In [67]:
tafe_resignations["Reasons for Leaving"] = tafe_resignations.apply(transformer, axis=1)

Now that both datasets have the same columns and the same possible values in each column, I can combine the two datasets.

### c) Combining the Datasets

Now I can combine the DETE and TAFE datasets together so they can be analyzed as a single dataset. First, I will create an "Institute" column that contains the name of the institute (either "DETE" or "TAFE") so I can distinguish between the two datasets if necessary.

In [68]:
# Combining the dataframes
dete_resignations["Institute"] = "DETE"
tafe_resignations["Institute"] = "TAFE"
combined_resignations = pd.concat([dete_resignations, tafe_resignations], ignore_index=True)
combined_resignations.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,Age,Dissatisfaction,Employment Status,Gender,Institute,Institute Tenure,Position,Reasons for Leaving
0,36-40,False,Full-time,Female,DETE,7-10,Teacher / TA,Career Move to Private Sector
1,41-45,True,Full-time,Female,DETE,11-20,Officer,"Career Move to Private Sector, Dissatisfaction..."
2,31-35,False,Full-time,Female,DETE,3-4,Teacher / TA,Career Move to Private Sector
3,46-50,True,Part-time,Female,DETE,11-20,Teacher / TA,"Dissatisfaction, Interpersonal Conflict"
4,31-35,False,Full-time,Male,DETE,3-4,Teacher / TA,"Maternal/Family, Relocation"


Next, I will save the combined_resignations dataset to a file so it can be referenced in the next section, which will be data cleaning.

In [69]:
combined_resignations.to_csv('combined_resignations.csv', index=False)